In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

In [ ]:
# Cargar el estado del modelo
model_path = "models/spanish_bert_cased.pth"
model_state_dict = torch.load(model_path,map_location=torch.device('cpu'))

In [ ]:
bert_model_name = 'dccuchile/bert-base-spanish-wwm-cased'

In [ ]:
# Inicializar un nuevo modelo BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(bert_model_name)

# Cargar el estado del modelo en el nuevo modelo
model.load_state_dict(model_state_dict)

# Asegúrate de poner el modelo en modo de evaluación
model.eval()

In [ ]:
def predict(text, model, tokenizer):
    # Tokenizar el texto de entrada
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Obtener las predicciones del modelo
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Obtener las probabilidades de las clases
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).squeeze().tolist()
    
    # Obtener la clase predicha
    predicted_class = torch.argmax(logits, dim=1).item()
    
    return predicted_class, probabilities

In [ ]:
def predict_veracity(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        print(outputs)
        _, preds = torch.max(outputs, dim=1)
        print(torch.max(outputs, dim=1))
        return "True" if preds.item() == 1 else "False"

In [ ]:
# Inicializar el tokenizer
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
# Texto de ejemplo para predecir
text = "This is a sample text for classification."

# Realizar la predicción
predicted_class, probabilities = predict(text, model, tokenizer)

print(f"Predicted class: {predicted_class}")
print(f"Probabilities: {probabilities}")